In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path="../../models/hf"
model_path="../../models/llama/hf"
model_path="../../models/llama/conv_hf"
model_path="../../models/hf/0503llama"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.92s/it]


In [4]:
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [5]:

tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [18]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=200, 
              repetition_penalty=1.2,
              temperature=0.6,
              )

In [20]:
import time
text_list=[
"Q. Generate Python code to generate fibonacci sequence.\n A. \n def fibonacci(n):"

]
text_list=[
"llamaのconvertは大変" ,   
"今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
]


for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

-------
input:  llamaのconvertは大変
perplexity:  1947.14599609375
time:  8.354416131973267
time/character:  0.01351847270545836
output:  llamaのconvertは大変です。
このように、Pythonコードを使用して解決できます。<llm-code>
import sympy as sp
from math import sqrt, pi

# Convert the number to a float and round it to two decimal places.
x = sp.Float(2) * (sp.pi/180)**3 + sp.Float(-4*sp.pi)/180**2 - x
rounded_x = round(x, 2)
print("The rounded value of x is:", rounded_x)
</llm-code><llm-code-output>
xの値は次のとおりです。9.0
</llm-code-output>
したがって、答えは\\ boxed {9}です #!/usr/bin/env python
"""
This script contains all the functions used in the main program. It also includes some useful utilities for working with data.
It can be run from command line or using Python's built-in shell.
"""
-------
input:  今日はいい
perplexity:  119.56906127929688
time:  8.555116176605225
time/character:  0.024654513477248487
output:  今日はいい人に見える。
「お前が俺の名前を言った時、俺は何て言うんだ」
「え?」
「いや、何でもない」
「じゃあ、なんで私と付き合うんですか!」
「だって、俺はお前のこと好きだよ」
「......っ!?」
「だから、俺はお前のことを嫌いなんだ

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-------
input:  Functional polymers are
perplexity:  85.88130950927734
time:  7.890818357467651
time/character:  0.008484750922008228
output:  Functional polymers are formed by the interaction of a single monomer with an adjacent monomer. Therefore, it is possible to predict the structure and properties of such systems using only one or two monomers. In this work we have used the 1D-NMR approach for the prediction of the structure and properties of the dimeric 2D-dimeric system ( Fig. 4a–c ). We have also investigated the effect of different oligomerization states on the structure and properties of the dimeric 2D-dimeric system. For example, in the case of the tetrameric 2D-dimeric system, we have found that the oligomerization state has no influence on the overall structure but significantly affects the oligomerization state. This result indicates that the oligomerization state can be considered as a key factor determining the oligomerization state of the protein.
Discussion
In order 

In [ ]:
tokenizer